In [55]:
from google.colab import drive
drive.mount('/content/drive')
%cd ./drive/My\ Drive/NLP-Model-Demo/RNN_Translator_Demo/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: './drive/My Drive/NLP-Model-Demo/RNN_Translator_Demo/'
/content/drive/My Drive/NLP-Model-Demo/RNN_Translator_Demo


In [56]:
from data_prepare import *

In [57]:
from models import Plain_RNN, Criterion

In [58]:
import importlib

In [59]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [60]:
device = 'cuda'

In [61]:
# Test data_prepare
train_en_w, train_cn_w = load_data(train_file)
dev_en_w, dev_cn_w = load_data(dev_file)

en2idx, idx2en = build_vocab(train_en_w)
cn2idx, idx2cn = build_vocab(train_cn_w)

train_en, train_cn = sentences2idx(train_en_w, train_cn_w, en2idx, cn2idx, len_sort=False)
dev_en, dev_cn = sentences2idx(dev_en_w, dev_cn_w, en2idx, cn2idx, len_sort=False)

In [62]:
batch_size = 64
train_data = gen_examples(train_en, train_cn, batch_size)
dev_data = gen_examples(dev_en, dev_cn, batch_size)
print('num of batchs:', len(train_data))
print('maxlen of No.200 batch:', train_data[200][0][0].size)

num of batchs: 228
maxlen of No.200 batch: 16


In [63]:
# choose No.200 batch
batch_x = torch.from_numpy(train_data[200][0]).long().to(device)
batch_x_len = torch.from_numpy(train_data[200][1]).long().to(device)
print('idx sentences:\n', batch_x, '\n', batch_x.shape)  # 64, 20
print('len of sentences:\n', batch_x_len, '\n', batch_x_len.shape)  # 64

idx sentences:
 tensor([[   2,   25,   10,  ...,    1,    1,    1],
        [   2,    5,   22,  ...,    1,    1,    1],
        [   2,   61,   28,  ...,    1,    1,    1],
        ...,
        [   2,   87,   14,  ...,    1,    1,    1],
        [   2,   28, 2193,  ...,    1,    1,    1],
        [   2,    5,  236,  ...,    1,    1,    1]], device='cuda:0') 
 torch.Size([64, 16])
len of sentences:
 tensor([ 7,  7,  9,  6,  9,  9, 10, 15, 10, 10,  7,  9, 14,  9, 10, 10,  9,  6,
         8, 11,  9, 10,  9,  8, 10, 16, 15,  9,  9,  7,  9,  9,  6,  8,  9, 10,
        10, 10, 11, 15,  7, 11,  8, 15,  7,  6, 10,  9,  7, 10,  7,  7,  9,  8,
        10,  6, 10, 11,  7,  9,  6, 10,  9,  6], device='cuda:0') 
 torch.Size([64])


In [64]:
# Test Encoder
importlib.reload(Plain_RNN)
M = Plain_RNN.PlainEncoder(len(en2idx), 100)
M.to(device)
output, hidden = M(batch_x, batch_x_len)
print(output.shape, hidden.shape)

torch.Size([64, 16, 100]) torch.Size([1, 64, 100])


In [65]:
dropout = 0.2
hidden_size=100
encoder = Plain_RNN.PlainEncoder(len(en2idx), hidden_size, dropout)
decoder = Plain_RNN.PlainDecoder(len(en2idx), hidden_size, dropout)

model = Plain_RNN.PlainSeq2Seq(encoder, decoder)
model = model.to(device)

loss_fn = Criterion.LanguageModelCriterion()
optimizer = torch.optim.Adam(model.parameters())

In [71]:
def train(model, data, num_epochs):
    model.train() # 训练模式
    for epoch in range(num_epochs):
        total_num_words = total_loss = 0.
        for it, (mb_x, mb_x_len, mb_y, mb_y_len) in enumerate(data):
            mb_x = torch.from_numpy(mb_x).to(device).long()
            mb_x_len = torch.from_numpy(mb_x_len).to(device).long()
            mb_input = torch.from_numpy(mb_y[:, :-1]).to(device).long() # EOS之前
            mb_output = torch.from_numpy(mb_y[:, 1:]).to(device).long() # BOS之后
            mb_y_len = torch.from_numpy(mb_y_len - 1).to(device).long()
            mb_y_len[mb_y_len <= 0] = 1
            mb_pred, attn = model(mb_x, mb_x_len, mb_input, mb_y_len)
            # [mb_y_len.max()]->[1, mb_y_len.max()]
            mb_out_mask = torch.arange(mb_y_len.max().item(), device=device)[None, :] < mb_y_len[:, None]
            mb_out_mask = mb_out_mask.float()
            # (pre, target, mask)
            # mb_output是句子单词的索引
            loss = loss_fn(mb_pred, mb_output, mb_out_mask)
            num_words = torch.sum(mb_y_len).item()
            total_loss += loss.item() * num_words
            total_num_words += num_words
            # 更新模型
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
            optimizer.step()
            if it % 100 == 0:
                print("Epoch: ", epoch, 'iteration', it, 'loss:', loss.item())
                # print("Epoch", epoch, "Training loss", total_loss / total_num_words)
            if epoch % 5 == 0:
                # evaluate(model, dev_data)
                torch.save(model.state_dict(), 'translate_model.pt')

In [72]:
train(model, train_data, num_epochs=100)

Epoch:  0 iteration 0 loss: 5.07315731048584
Epoch:  0 iteration 100 loss: 4.649954795837402
Epoch:  0 iteration 200 loss: 4.615468502044678
Epoch:  1 iteration 0 loss: 4.459018230438232
Epoch:  1 iteration 100 loss: 4.126104354858398
Epoch:  1 iteration 200 loss: 4.121497631072998
Epoch:  2 iteration 0 loss: 4.008650779724121
Epoch:  2 iteration 100 loss: 3.795247793197632
Epoch:  2 iteration 200 loss: 3.8154125213623047
Epoch:  3 iteration 0 loss: 3.7629566192626953
Epoch:  3 iteration 100 loss: 3.593043088912964
Epoch:  3 iteration 200 loss: 3.6101157665252686
Epoch:  4 iteration 0 loss: 3.5578391551971436
Epoch:  4 iteration 100 loss: 3.4303364753723145
Epoch:  4 iteration 200 loss: 3.4314322471618652
Epoch:  5 iteration 0 loss: 3.4152586460113525
Epoch:  5 iteration 100 loss: 3.295196294784546
Epoch:  5 iteration 200 loss: 3.277961254119873
Epoch:  6 iteration 0 loss: 3.33132004737854
Epoch:  6 iteration 100 loss: 3.2044777870178223
Epoch:  6 iteration 200 loss: 3.167145013809204
